In [13]:
import os

# Create src directory if it doesn't exist
os.makedirs("src", exist_ok=True)
print("src/ directory is ready.")


src/ directory is ready.


In [14]:
constants_code = """
# Constants used throughout the simulation

daily_installs = 20000
arppu = 5.0

purchase_rate_A = 0.0305
purchase_rate_B = 0.0315

impressions_per_dau_A = 2.3
impressions_per_dau_B = 1.6

ecpm_A = 9.80
ecpm_B = 10.80
"""

with open("src/constants.py", "w") as f:
    f.write(constants_code.strip())

print("constants.py created.")


constants.py created.


In [16]:
retention_models_code = """
import numpy as np

# Base retention models
def exp(x, a, b):
    return a * np.exp(-b * (x - 1))

def power(x, a, b):
    return a * x ** (-b)

def log(x, a, b):
    return a - b * np.log(x)

def inv(x, a, b):
    return a / (x + b)

# Task 1e – New user source custom curves
def new_retention_A(age):
    return max(0.5 * np.exp(-0.25 * (age - 1)), 0)

def new_retention_B(age):
    return max(0.45 * np.exp(-0.22 * (age - 1)), 0)
"""

with open("src/retention_models.py", "w") as f:
    f.write(retention_models_code.strip())

print("retention_models.py created.")


retention_models.py created.


In [17]:
simulation_code = """
def simulate_dau(days, params, daily_installs, retention_func):
    \"""
    Simulates daily active users (DAU) over a number of days for a given retention model.
    \"""
    dau = []
    for day in range(1, days + 1):
        active = 0
        for cohort_day in range(1, day + 1):
            age = day - cohort_day + 1
            active += retention_func(age, *params) * daily_installs
        dau.append(active)
    return dau

def calc_revenue(dau, purchase_rate, impressions_per_dau, ecpm, arppu):
    ad_rev = sum([d * impressions_per_dau * ecpm / 1000 for d in dau])
    iap_units = sum([d * purchase_rate for d in dau])
    iap_rev = iap_units * arppu
    total_rev = ad_rev + iap_rev
    return ad_rev, iap_rev, total_rev
"""

with open("src/simulation.py", "w") as f:
    f.write(simulation_code.strip())

print("simulation.py created.")


simulation.py created.


In [15]:
modeling_code = """
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

def fit_and_score_models(models, x, yA, yB, daily_installs):
    summary_rows = []
    params_dict = {}

    for name, f, bounds in models:
        pa, _ = curve_fit(f, x, yA, bounds=bounds)
        pb, _ = curve_fit(f, x, yB, bounds=bounds)

        r2a = r2_score(yA, f(x, *pa))
        r2b = r2_score(yB, f(x, *pb))
        avg_r2 = np.mean([r2a, r2b])

        params_dict[name] = {'A': pa, 'B': pb, 'func': f}

        ages = np.arange(1, 16)
        dau_a = np.sum(f(ages, *pa) * daily_installs)
        dau_b = np.sum(f(ages, *pb) * daily_installs)

        summary_rows.append([name, round(r2a, 3), round(r2b, 3), round(avg_r2, 3), round(dau_a), round(dau_b)])

    summary = pd.DataFrame(summary_rows, columns=["Model", "R² A", "R² B", "Avg R²", "DAU A", "DAU B"])
    summary = summary.sort_values("Avg R²", ascending=False).reset_index(drop=True)
    return summary, params_dict

def plot_retention_curves(params_dict, x, yA, yB, horizons=[15, 30, 180, 360]):
    x_smooth = {h: np.linspace(1, h, 500) for h in horizons}
    fig, axs = plt.subplots(len(horizons), 2, figsize=(14, 3.5 * len(horizons)))
    axs = axs if axs.ndim == 2 else axs.reshape(len(horizons), 2)

    for i, horizon in enumerate(horizons):
        xs = x_smooth[horizon]
        for j, variant in enumerate(["A", "B"]):
            ax = axs[i, j]
            for name in params_dict:
                f = params_dict[name]['func']
                p = params_dict[name][variant]
                y_pred = f(xs, *p)
                ax.plot(xs, y_pred, label=name)
            
            ax.scatter(x, yA if variant == "A" else yB, marker='o', color='black', label='Observed')
            ax.set_title(f"{variant} - Retention up to Day {horizon}")
            ax.set_ylim(-0.1, 1.2)
            ax.set_xlabel("Day")
            ax.set_ylabel("Retention")
            ax.legend(loc='upper right')
            ax.grid(True)

    plt.tight_layout()
    plt.show()
"""

with open("src/modeling.py", "w") as f:
    f.write(modeling_code.strip())

print("modeling.py created.")


modeling.py created.
